In [1]:
%matplotlib inline  
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_rcv1
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.cluster import k_means

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def load_ks(path_to_ks):
    ks = pd.read_csv(path_to_ks, names=['k'], dtype=np.int32)
    return ks['k'].tolist()

In [3]:
def check_output(output, y):
    assert type(output) == np.ndarray
    assert output.ndim == 1
    assert output.shape[0] == y.shape[0]

In [5]:
# Adrien Cogny - Assignment 1 for Data Science in the Wild at Cornell Tech
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_rcv1
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

def load_labels(pathtoLabels):
    DF = pd.read_csv(pathtoLabels,header=None);
    return DF[0];

def load_training_data():
    dataSet = fetch_rcv1(subset='train')

    return dataSet.data, dataSet.target, dataSet.sample_id

def load_validation_data(validationData):

    # This is taken from the homework FAQ and modified slightly.
    test_data = fetch_rcv1(subset='test')
    ids = pd.read_csv(validationData)
    mask = np.isin(test_data.sample_id, ids)
    validation_data = test_data.data[mask]
    validation_target = test_data.target[mask]
    return validation_data, validation_target, test_data

class CS5304KNNClassifier():

    numNeighbors = 5;

    classifier = None;
    # def __init__(self,):
    #     # Initialization of the knn classifier should be done here
    #     self.classifier = NearestNeighbors(n_neighbors=self.numNeighbors);        
    #     return
    def __init__(self,n_neighbors = None):
        if (type(n_neighbors) == int):
            self.numNeighbors = n_neighbors;
        self.classifier = KNeighborsClassifier(n_neighbors=self.numNeighbors,algorithm='brute');        

    def train(self,fitX, fitY):
        #in the fitX, fitY sparce matrix, the index of the element is in indptr

        fitXIndexElem = fitX.indptr;
        fitYIndexElem = fitY.indptr;

        fitXelemDF = pd.Series(data=fitX.indptr[1:])
        fitYelemDF = pd.Series(data=fitY.indptr[1:])
        # print(fitXelemDF);
        # print(fitYelemDF);


        dataArray = fitX.toarray()
        dataDF = pd.DataFrame(data=dataArray,index=fitXelemDF)
        # print(dataDF)

        targetArray = fitY.toarray()
        targetDF = pd.DataFrame(data=targetArray,index=fitYelemDF)
        # print(targetDF)

        self.classifier.fit(dataDF,targetDF);
        return

    def predict(self,predictX):
        dataArray = predictX.toarray()
        dataDF = pd.DataFrame(data=dataArray)
        return self.classifier.predict(predictX)

    def score(self,data,labels):
        dataArray = data.toarray()
        dataDF = pd.DataFrame(data=dataArray)

        predictArray = labels.toarray()
        labelsDF = pd.DataFrame(data=predictArray)

        return self.classifier.score(dataDF,labelsDF)



class CS5304NBClassifier():

    classifier = None;
    def __init__(self, alphanew = 1.25):
        self.classifier = BernoulliNB(alpha = alphanew)
        return
    
    def train(self,fitX, fitY):
        
        self.classifier.fit(fitX,fitY.todense())
        return

    def predict(self,predictX):
        dataArray = predictX.toarray()
        dataDF = pd.DataFrame(data=dataArray)
        return self.classifier.predict(predictX)

    def score(self,data,labels):
        return self.classifier.score(data,labels.todense())

class CS5304KMeansClassifier():
    classifier = None;
#     centroids = None;
    numClusters = 2;
    def __init__(self, n_clusters=2, init='k-means++'):
        self.numClusters = n_clusters
        return
    
    def train(self,fitX, fitY):
        dataset = pd.DataFrame(data = fitX.toarray())
        dataset['label'] = fitY.todense()
        print(dataset.head(20))
#         dataset.info(memory_usage='deep')
        
#         meanData = dataset[dataset['label'] == 1].drop(columns = ['label']).mean()
#         meanOtherData = dataset[dataset['label'] == 0].drop(columns = ['label']).mean()
        preClassifier = KMeans(n_clusters=1)
        preClassifier.fit(dataset[dataset['label'] == 1].drop(columns = ['label']))
        
        meanCentroid = pd.Series(preClassifier.cluster_centers_[0])
#         print(meanCentroid);
        
        preClassifier.fit(dataset[dataset['label'] == 0].drop(columns = ['label']))
        meanOtherDataCentroid = pd.Series(preClassifier.cluster_centers_[0])
#         print(meanOtherDataCentroid);
        
        
        
        
#         print(meanData)
#         print(meanData.shape)
#         print(meanOtherData)
#         print(meanOtherData.shape)
        
        del dataset # memory drain so delete it
        
        initClusters = pd.concat([meanOtherDataCentroid, meanCentroid], axis = 1);
        
        print(initClusters.head())
        print(initClusters.shape)
        
        
        
        self.classifier = KMeans(n_clusters=self.numClusters, init=initClusters.T)

        self.classifier.fit(fitX,fitY.todense())
        
        return 
    
    def predict(self,predictX):
#         dataArray = predictX.toarray()
#         dataDF = pd.DataFrame(data=dataArray)
        return self.classifier.predict(predictX)

    def score(self,data,labels):
        predictedClusters = self.classifier.predict(data);
        dataFrame = pd.DataFrame(data = predictedClusters, columns=['Predicted'])
        dataFrame['actual'] = labels.todense()
        dataFrame['predictedAccuracy'] = dataFrame['Predicted'] == dataFrame['actual']
        
        numItemsOfClass = (dataFrame['actual'] == 1).sum()
        numItemsNotOfClass = (dataFrame['actual'] == 0).sum()
        
        truePositives = ((dataFrame['actual'] == 1) & (dataFrame['Predicted'] == 1)).sum()
        falsePositives = ((dataFrame['actual'] == 0) & (dataFrame['Predicted'] == 1)).sum()
        falseNegatives = ((dataFrame['actual'] == 1) & (dataFrame['Predicted'] == 0)).sum()
        trueNegatives = ((dataFrame['actual'] == 0) & (dataFrame['Predicted'] == 0)).sum()
        
        Precision = truePositives / (truePositives + falsePositives)
        Recall = truePositives / (truePositives + falseNegatives)
        print(Precision)
        print(Recall)
        f1 = 1/((1/Precision) + (1/Recall))
        return f1

In [5]:
path_to_labels = "labels.txt"
path_to_ids = "validation.txt"
path_to_ks = "ks.txt"
label = 33

In [6]:
ks = load_ks(path_to_ks)
labels = load_labels(path_to_labels)
train_data, train_target, _ = load_training_data()
eval_data, eval_target, _ = load_validation_data(path_to_ids)

In [7]:
print(labels)

0      4
1      5
2      7
3     33
4     59
5     70
6     83
7     95
8     98
9    102
Name: 0, dtype: int64


# KNN Classifier Testing

In [ ]:
scoresDF = pd.DataFrame(index=[i for i in range(1,21)],columns=[label for label in labels])
for label in labels:    
    print(label)
    # Grade Ex. 1a
    for k in range(1,21):
#         print(k)
        limit = 1000
#         k = 15 # change this
        knn = CS5304KNNClassifier(n_neighbors=k)
        knn.train(train_data[:limit], train_target[:limit][:, label])
        output = knn.predict(eval_data[:limit])
        check_output(output,eval_target[:limit])
    #     print(knn.score(eval_data[:limit],eval_target[:limit][:, label]))
        scoresDF[label][k] = knn.score(eval_data[:limit],eval_target[:limit][:, label])

scoresDF.head()

In [ ]:
scoresDF.head(20)
scoresDF.to_csv('scores_matrix')

In [ ]:
scoresDF.max()

In [ ]:
scoresDF.head(20)

In [ ]:
scoresDF.plot()

# NB Classification Testing

In [ ]:
# print(label)
# # nb = CS5304NBClassifier()
# # nb.train(train_data, train_target[:, label])
# # output = nb.predict(eval_data)
# # check_output(output,eval_target)
# # print(nb.score(eval_data,eval_target[:, label]))

scoresNBDF = pd.DataFrame(index=[label for label in labels],columns=[i for i in np.arange(0.1, 2.0, 0.1)])
for label in labels:
    print(label)
    for alpha in np.arange(0.1, 2.0, 0.1):  
#         print(alpha)
        nb = CS5304NBClassifier(alpha)
        nb.train(train_data, train_target[:, label])
        output = nb.predict(eval_data)
        check_output(output,eval_target)
    #     print(nb.score(eval_data,eval_target[:, label]))
        scoresNBDF[alpha][label] = nb.score(eval_data,eval_target[:, label])
scoresNBDF.head()

In [ ]:
scoresNBDF.head(20)

In [ ]:
scoresNBDF.T.plot()

# Kmeans Clustering

In [ ]:
kmeans = CS5304KMeansClassifier()
kmeans.train(train_data, train_target[:, label])

In [ ]:
checkingClassifier = kmeans.predict(train_data);
pd.DataFrame(data=checkingClassifier)
kmeans.score(eval_data,eval_target[:, label])

In [ ]:
output = kmeans.predict(eval_data)
check_output(output,eval_target)

In [ ]:
score = kmeans.score(eval_data,eval_target[:, label])

In [ ]:
print(score)